In [178]:
import torch
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import ClusterCentroids
from scipy.stats import chi2_contingency
from sklearn import metrics

# 所有变量格式化
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [179]:
spdata = pd.read_pickle('SP.pkl')
mpdata = pd.read_pickle('MP.pkl')

In [180]:
# 指定标签

spdata['label'] = 0
mpdata['label'] = 1

In [181]:
# 提取共有特征

intersection_columns = list(set(spdata.columns).intersection(set(mpdata.columns)))

In [182]:
# 合并单双原发癌表

mixdata = pd.DataFrame()
# mixdata = mixdata.append(spdata[intersection_columns].sample(n=8288, random_state=1))
mixdata = mixdata.append(spdata[intersection_columns])
mixdata = mixdata.append(mpdata[mpdata['Record number recode']==1][intersection_columns])

In [183]:
# 标记 p < 0.05
def highlight_diff(rowdata):
    return ['background-color: darkgreen' if rowdata['p'] < 0.05 else '' for v in rowdata]

In [184]:
# 对照spdata, mpdata交集列显示统计信息
def columninfo(tag, spdata, mpdata):

    print('='*90)

    # 分项统计
    svalue = spdata.groupby(tag).size().sort_values(ascending=False)
    print('spdata:', svalue)
    print('-'*90)    
    mvalue = mpdata[mpdata['Record number recode']==1].groupby(tag).size().sort_values(ascending=False)
    print('mpdata:', mvalue)
    print('-'*90)

    # 项目列表
    snunique = spdata[tag].nunique()
    mnunique = mpdata[mpdata['Record number recode']==1][tag].nunique()
    print(f'spdata: {snunique}, mpdata: {mnunique}')
    sunique = spdata[tag].unique()
    munique = mpdata[mpdata['Record number recode']==1][tag].unique()
    index = list(set(sunique).intersection(set(munique)))
    print('difference:\n')
    sdiff = list(set(sunique).difference(set(munique)))
    print('spdata: ', sdiff)
    mdiff = list(set(munique).difference(set(sunique)))
    print('mpdata: ', mdiff)
    print('*'*90)

    # 同项目卡方检验，排序时用'0'，排序后换回0
    obs = np.array([svalue[index], mvalue[index]]).T

    obsframe = pd.DataFrame(index=index, columns=['svalue','mvalue','percent'])
    obsframe['svalue'] = obs[:,0]
    obsframe['mvalue'] = obs[:,1]
    obsframe['percent'] = round(obsframe['mvalue']/(obsframe['svalue']+obsframe['mvalue']), 3)
    print(obsframe.sort_values(by='percent', ascending=False))
    
    # 整体差异性
    print('p = ', round(chi2_contingency(obs)[1], 3))
    print('-'*90)

    # 组间两两比较
    result = pd.DataFrame()
    result_type1 = []
    result_percent1 = []
    result_type2 = []
    result_percent2 = []
    result_p = []
    for type1 in range(len(obs)):
        for type2 in range(type1+1, len(obs)):
            result_type1.append(index[type1])
            result_percent1.append(obsframe.loc[index[type1], 'percent'])
            result_type2.append(index[type2])
            result_percent2.append(obsframe.loc[index[type2], 'percent'])
            result_p.append(round(chi2_contingency(np.array([obs[type1], obs[type2]]))[1],3))
    result['type1'] = result_type1
    result['percent1'] = result_percent1
    result['type2'] = result_type2
    result['percent2'] = result_percent2
    result['p'] = result_p

    return result.style.apply(highlight_diff, axis=1).format({'p': '{:.3f}', 'percent1': '{:.3f}', 'percent2': '{:.3f}'})

In [185]:
mixdata.columns

Index(['Radiation sequence with surgery', 'Derived AJCC N, 7th ed (2010-2015)',
       'SEER Combined Mets at DX-liver (2010+)',
       'Derived SEER Combined N (2016+)', 'Derived SEER Combined T (2016+)',
       'Radiation recode', 'Tumor Size Summary (2016+)',
       'RX Summ--Surg Prim Site (1998+)', 'Year of diagnosis',
       'Derived SEER Combined M (2016+)',
       'SEER Combined Mets at DX-lung (2010+)',
       'Derived SEER Cmb Stg Grp (2016+)', 'Breast Subtype (2010+)',
       'Derived AJCC M, 7th ed (2010-2015)',
       'Derived AJCC T, 7th ed (2010-2015)', 'Histologic Type ICD-O-3',
       'Marital status at diagnosis', 'CS tumor size (2004-2015)',
       'Site recode ICD-O-3/WHO 2008', 'Age at diagnosis',
       'COD to site recode', 'Sequence number',
       'CS site-specific factor 7 (2004+ varying by schema)', 'label', 'Grade',
       'Derived HER2 Recode (2010+)', 'Chemotherapy recode (yes, no/unk)',
       'SEER Combined Mets at DX-brain (2010+)',
       'SEER Combine

In [186]:
mixdata.shape

(323435, 37)

In [187]:
# 指定标签及特征

data = mixdata[['label', 'Age at diagnosis', 'Derived AJCC Stage Group, 7th ed (2010-2015)', 'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)', 'Derived AJCC M, 7th ed (2010-2015)', 'Grade', 'Breast Subtype (2010+)', 'ER Status Recode Breast Cancer (1990+)', 'PR Status Recode Breast Cancer (1990+)', 'Derived HER2 Recode (2010+)']]

In [188]:
data = data.dropna()

In [189]:
# Age按照50岁分组

# data['Age'] = pd.cut(data['Age at diagnosis'], bins=[0,50,200], right=False)
data['Age'] = data['Age at diagnosis']
data = data.drop(columns='Age at diagnosis')

In [190]:
# 去掉Stage中无意义项目

data = data.drop(data[data['Derived AJCC Stage Group, 7th ed (2010-2015)'].isin(['Blank(s)', 'UNK Stage', 0, 'IIINOS'])].index)
data = data.rename(columns={'Derived AJCC Stage Group, 7th ed (2010-2015)': 'Stage'})

In [191]:
# 去掉T无意义项目

data = data.drop(data[data['Derived AJCC T, 7th ed (2010-2015)'].isin(['T0', 'T1mic', 'TX', 'T4NOS', 'T1NOS'])].index)
data = data.rename(columns={'Derived AJCC T, 7th ed (2010-2015)': 'T'})

In [192]:
# 去掉N无意义项目

data = data.drop(data[data['Derived AJCC N, 7th ed (2010-2015)'].isin(['N1mi', 'N2NOS', 'N1NOS', 'N0(mol-)', 'N3NOS', 'NX', 'N0(mol+)'])].index)
data = data.rename(columns={'Derived AJCC N, 7th ed (2010-2015)': 'N'})

In [193]:
# 去掉M1

data = data.drop(data[data['Derived AJCC M, 7th ed (2010-2015)'].isin(['M1', 'M0(i+)'])].index)
data = data.drop(columns='Derived AJCC M, 7th ed (2010-2015)')

In [194]:
# 去掉Grade无意义项目

data = data.drop(data[data['Grade'].isin(['Unknown'])].index)

In [195]:
# 去掉Subtype无意义项目

data = data.drop(data[data['Breast Subtype (2010+)'].isin(['Unknown'])].index)
data = data.rename(columns={'Breast Subtype (2010+)': 'Subtype'})

In [196]:
# 去掉ER无意义项目

data = data.drop(data[data['ER Status Recode Breast Cancer (1990+)'].isin(['Unknown', 'Borderline'])].index)
data = data.rename(columns={'ER Status Recode Breast Cancer (1990+)': 'ER'})

In [197]:
# 去掉PR无意义项目

data = data.drop(data[data['PR Status Recode Breast Cancer (1990+)'].isin(['Unknown', 'Borderline'])].index)
data = data.rename(columns={'PR Status Recode Breast Cancer (1990+)': 'PR'})

In [198]:
# 去掉HER2无意义项目

data = data.drop(data[data['Derived HER2 Recode (2010+)'].isin(['Unknown', 'Borderline'])].index)
data = data.rename(columns={'Derived HER2 Recode (2010+)': 'HER2'})

In [199]:
data

,label,Stage,T,N,Grade,Subtype,ER,PR,HER2,Age
1,0,IIB,T2,N1a,Moderately differentiated; Grade II,HR+/HER2- (Luminal A),Positive,Positive,Negative,53
2,0,IIB,T2,N1a,Poorly differentiated; Grade III,HR+/HER2- (Luminal A),Positive,Negative,Negative,59
6,0,IA,T1c,N0,Moderately differentiated; Grade II,HR+/HER2- (Luminal A),Negative,Positive,Negative,89
11,0,IIIA,T1c,N2a,Poorly differentiated; Grade III,HR+/HER2- (Luminal A),Positive,Positive,Negative,62
12,0,IIIA,T2,N2a,Moderately differentiated; Grade II,HR+/HER2- (Luminal A),Positive,Positive,Negative,65
...,...,...,...,...,...,...,...,...,...,...
16214,1,IIA,T1b,N1a,Poorly differentiated; Grade III,HR-/HER2- (Triple Negative),Negative,Negative,Negative,68
16264,1,IA,T1b,N0(i-),Well differentiated; Grade I,HR+/HER2- (Luminal A),Positive,Positive,Negative,70
16354,1,IA,T1c,N0(i-),Poorly differentiated; Grade III,HR+/HER2- (Luminal A),Positive,Positive,Negative,65
16452,1,IIA,T2,N0,Poorly differentiated; Grade III,HR-/HER2- (Triple Negative),Negative,Negative,Negative,64


In [200]:
labels = data['label']

features = data[['Age', 'Stage', 'T', 'N', 'Grade', 'Subtype', 'ER', 'PR', 'HER2']]

In [201]:
# One-Hot 编码分类数据

features = pd.get_dummies(features)

In [202]:
features

,Age,Stage_IA,Stage_IIA,Stage_IIB,Stage_IIIA,Stage_IIIB,Stage_IIIC,T_T1a,T_T1b,T_T1c,...,Subtype_HR+/HER2+ (Luminal B),Subtype_HR+/HER2- (Luminal A),Subtype_HR-/HER2+ (HER2 enriched),Subtype_HR-/HER2- (Triple Negative),ER_Negative,ER_Positive,PR_Negative,PR_Positive,HER2_Negative,HER2_Positive
1,53,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
2,59,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,1,0
6,89,1,0,0,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,1,0
11,62,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,1,0,1,1,0
12,65,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16214,68,0,1,0,0,0,0,0,1,0,...,0,0,0,1,1,0,1,0,1,0
16264,70,1,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,1,1,0
16354,65,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,0,1,1,0
16452,64,0,1,0,0,0,0,0,0,0,...,0,0,0,1,1,0,1,0,1,0


In [203]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=1, stratify=labels)

In [204]:
from imblearn.under_sampling import NearMiss

nm3 = NearMiss(version=3)

x_resample, y_resample = nm3.fit_resample(x_train, y_train)

In [205]:
# 数值标准化

# features = preprocessing.StandardScaler().fit_transform(features)

In [206]:
x_test['label'] = y_test


In [207]:
testframe = pd.DataFrame()
testframe = testframe.append(x_test[x_test['label']==1].sample(n=500, random_state=2))
testframe = testframe.append(x_test[x_test['label']==0].sample(n=500, random_state=2))
testframe

,Age,Stage_IA,Stage_IIA,Stage_IIB,Stage_IIIA,Stage_IIIB,Stage_IIIC,T_T1a,T_T1b,T_T1c,...,Subtype_HR+/HER2- (Luminal A),Subtype_HR-/HER2+ (HER2 enriched),Subtype_HR-/HER2- (Triple Negative),ER_Negative,ER_Positive,PR_Negative,PR_Positive,HER2_Negative,HER2_Positive,label
12310,81,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,1,1
7072,59,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,1,0,1
2404,87,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,1,0,1
5672,63,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,1,0,1
3058,74,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55909,79,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,1,0,0
256293,62,1,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,1,0,0
12293,63,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,1,0,0
256062,58,0,0,1,0,0,0,0,0,0,...,1,0,0,0,1,1,0,1,0,0


In [208]:
y_test = testframe['label']
x_test = testframe.drop(columns='label')

In [209]:
torch.cuda.is_available()

True

In [210]:
# 指定 x, y

x_train = torch.from_numpy(np.array(x_resample)).float().cuda()
y_train = torch.from_numpy(np.array(y_resample)).cuda()

In [211]:
# 构建网络

net = torch.nn.Sequential(
    torch.nn.Linear(x_train.shape[1], 200),
    torch.nn.Sigmoid(),
    torch.nn.Linear(200, 2)
)

optimizer = torch.optim.SGD(net.parameters(), lr=0.5)
loss_func = torch.nn.CrossEntropyLoss()

In [212]:
net = net.cuda()

In [213]:
x_train.shape

torch.Size([7262, 42])

In [214]:
y_train.shape

torch.Size([7262])

In [215]:
for t in range(10000):
    prediction = net(x_train)
    loss = loss_func(prediction, y_train)

    if t%500==0:
        print(t, loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 tensor(0.7190, device='cuda:0', grad_fn=<NllLossBackward>)
500 tensor(0.6908, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.6826, device='cuda:0', grad_fn=<NllLossBackward>)
1500 tensor(0.6815, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.6867, device='cuda:0', grad_fn=<NllLossBackward>)
2500 tensor(0.6891, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.7924, device='cuda:0', grad_fn=<NllLossBackward>)
3500 tensor(0.7126, device='cuda:0', grad_fn=<NllLossBackward>)
4000 tensor(0.6965, device='cuda:0', grad_fn=<NllLossBackward>)
4500 tensor(0.7004, device='cuda:0', grad_fn=<NllLossBackward>)
5000 tensor(0.7694, device='cuda:0', grad_fn=<NllLossBackward>)
5500 tensor(0.8434, device='cuda:0', grad_fn=<NllLossBackward>)
6000 tensor(0.7538, device='cuda:0', grad_fn=<NllLossBackward>)
6500 tensor(0.7572, device='cuda:0', grad_fn=<NllLossBackward>)
7000 tensor(0.7624, device='cuda:0', grad_fn=<NllLossBackward>)
7500 tensor(0.6907, device='cuda:0', grad_fn

In [216]:
y_predict = torch.max(torch.nn.functional.softmax(net(torch.from_numpy(np.array(x_test)).float().cuda())),1)[1]
matrix = confusion_matrix(y_test, y_predict.cpu())
matrix

array([[500,   0],
       [500,   0]], dtype=int64)

In [217]:
classification_report = metrics.classification_report(y_test, y_predict.cpu())
print(classification_report)

              precision    recall  f1-score   support

           0       0.50      1.00      0.67       500
           1       0.00      0.00      0.00       500

    accuracy                           0.50      1000
   macro avg       0.25      0.50      0.33      1000
weighted avg       0.25      0.50      0.33      1000



In [218]:
# from pycaret.classification import *

In [219]:
# # mldata = pd.DataFrame()
# mldata = mldata.append(data[data['label']==0])
# mldata = mldata.append(data[data['label']==1])

In [220]:
# clf = setup(mldata, target='label')

In [221]:
# best_model = compare_models()